In [ ]:
!pip install scipy==0.19.1   #from updated versions of scipy imresize is removed
!pip install tensorflow==1.15.0
!pip install keras==2.2.4


     |████████████████████████████████| 48.2MB 59kB/s 
ERROR: yellowbrick 0.9.1 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 0.19.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 0.19.1 which is incompatible.
ERROR: seaborn 0.10.1 has requirement scipy>=1.0.1, but you'll have scipy 0.19.1 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: librosa 0.6.3 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: cvxpy 1.0.31 has requirement scipy>=1.1.0, but you'll have scipy 0.19.1 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1

In [ ]:
#utils, processes images

from keras.layers import Lambda
import tensorflow as tf
import numpy as np
from numpy import array
from numpy.random import randint
from scipy.misc import imresize
import os
import sys
import imageio

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.switch_backend('agg')



# Subpixel Conv will upsample from (h, w, c) to (h/r, w/r, c/r^2)
def SubpixelConv2D(input_shape, scale=4):
    def subpixel_shape(input_shape):
        dims = [input_shape[0],input_shape[1] * scale,input_shape[2] * scale,int(input_shape[3] / (scale ** 2))]
        output_shape = tuple(dims)
        return output_shape
    
    def subpixel(x):
        return tf.depth_to_space(x, scale)
        
    return Lambda(subpixel, output_shape=subpixel_shape)
    
# Takes list of images and provide HR images in form of numpy array
def hr_images(images):
    images_hr = array(images)
    return images_hr

# Takes list of images and provide LR images in form of numpy array
def lr_images(images_real , downscale):
    
    images = []
    for img in  range(len(images_real)):
        images.append(imresize(images_real[img], [images_real[img].shape[0]//downscale,images_real[img].shape[1]//downscale], interp='bicubic', mode=None))
    images_lr = array(images)
    return images_lr
    
def normalize(input_data):

    return (input_data.astype(np.float32) - 127.5)/127.5 
    
def denormalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8)

def load_path(path):
    directories = []
    if os.path.isdir(path):
        directories.append(path)
    for elem in os.listdir(path):
        if os.path.isdir(os.path.join(path,elem)):
            directories = directories + load_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    return directories
    
def load_data_from_dirs(dirs, ext):
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in os.listdir(d): 
            if f.endswith(ext):
                image = imageio.imread(os.path.join(d,f))
                if len(image.shape) > 2:
                    files.append(imresize(image, (320, 240)))
                    file_names.append(os.path.join(d,f))
                count = count + 1
    return files     

def load_data(directory, ext):

    files = load_data_from_dirs(load_path(directory), ext)
    return files

def load_training_data(directory, ext, number_of_images = 1000, train_test_ratio = 0.8):

    number_of_train_images = int(number_of_images * train_test_ratio)
    
    files = load_data(directory, ext)
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
        
    test_array = array(files)
    if len(test_array.shape) < 3:
        print("Images are of not same shape")
        print("Please provide same shape images")
        sys.exit()
    
    x_train = files[:number_of_train_images]
    x_test = files[number_of_train_images:number_of_images]
    
    x_train_hr = hr_images(x_train)
    x_train_hr = normalize(x_train_hr)
    
    x_train_lr = lr_images(x_train, 4)
    x_train_lr = normalize(x_train_lr)
    
    x_test_hr = hr_images(x_test)
    x_test_hr = normalize(x_test_hr)
    
    x_test_lr = lr_images(x_test, 4)
    x_test_lr = normalize(x_test_lr)
    
    return x_train_lr, x_train_hr, x_test_lr, x_test_hr


def load_test_data_for_model(directory, ext, number_of_images = 100):

    files = load_data(directory, ext)
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()

    rand_nums = np.random.randint(0, files.shape[0], size=number_of_images)
        
    x_test_hr = hr_images(files[rand_nums])
    x_test_hr = normalize(x_test_hr)
    
    x_test_lr = lr_images(files[rand_nums], 4)
    x_test_lr = normalize(x_test_lr)
    
    return x_test_lr, x_test_hr
    
def load_test_data(directory, ext, number_of_images = 100):

    files = load_data(directory, ext)
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()

    rand_nums = np.random.randint(0, files.shape[0], size=number_of_images)
        
    x_test_lr = lr_images(files[rand_nums], 4)
    x_test_lr = normalize(x_test_lr)
    
    return x_test_lr
    

def plot_generated_images(output_dir, epoch, generator, x_test_hr, x_test_lr , dim=(1, 3), figsize=(15, 5), batch_size = 8):
    
    batch_count = int(x_test_hr.shape[0] / batch_size)

    generated_image = []

    for k in range(batch_count):

      lo = batch_size*k
      hi = batch_size*(k+1)
      if(hi>x_test_hr.shape[0]):
        hi = x_test_hr.shape[0]

      image_batch_lr_here = x_test_lr[lo:hi]
      gen_img = generator.predict(image_batch_lr_here)
      generated_image.append(denormalize(gen_img))
      #print(k, ' ', lo, ' ', hi, ' ', image_batch_lr_here.shape[0], ' ', gen_img.shape[0], ' ', len(generated_image))



    examples = x_test_hr.shape[0]
    #print(examples)
    #print(len(generated_image))
    value = randint(0, examples)
    value_i = value // 8
    value_j = value % 8
    image_batch_hr = denormalize(x_test_hr)
    image_batch_lr = denormalize(x_test_lr)
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_lr[value], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[value_i][value_j], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_hr[value], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(output_dir + 'generated_image_%d.png' % epoch)
    
    #plt.show()
    

def plot_test_generated_images_for_model(output_dir, generator, x_test_hr, x_test_lr , dim=(1, 3), figsize=(15, 5), batch_size = 8):
    
    batch_count = int(x_test_hr.shape[0] / batch_size)

    generated_image = []

    for k in range(batch_count):

      lo = batch_size*k
      hi = batch_size*(k+1)
      if(hi>x_test_hr.shape[0]):
        hi = x_test_hr.shape[0]

      image_batch_lr_here = x_test_lr[lo:hi]
      gen_img = generator.predict(image_batch_lr_here)
      generated_image.append(denormalize(gen_img))

    examples = x_test_hr.shape[0]
    image_batch_hr = denormalize(x_test_hr)
    image_batch_lr = denormalize(x_test_lr)
    
    for index in range(examples):

        value_i = index // 8
        value_j = index % 8
    
        plt.figure(figsize=figsize)
    
        plt.subplot(dim[0], dim[1], 1)
        plt.imshow(image_batch_lr[index], interpolation='nearest')
        plt.axis('off')
        
        plt.subplot(dim[0], dim[1], 2)
        plt.imshow(generated_image[index_i][index_j], interpolation='nearest')
        plt.axis('off')
    
        plt.subplot(dim[0], dim[1], 3)
        plt.imshow(image_batch_hr[index], interpolation='nearest')
        plt.axis('off')
    
        plt.tight_layout()
        plt.savefig(output_dir + 'test_generated_image_%d.png' % index)
    
        #plt.show()


def plot_test_generated_images(output_dir, generator, x_test_lr, figsize=(5, 5), batch_size = 8):
    
    batch_count = int(x_test_lr.shape[0] / batch_size)

    generated_image = []

    for k in range(batch_count):

      lo = batch_size*k
      hi = batch_size*(k+1)
      if(hi>x_test_lr.shape[0]):
        hi = x_test_lr.shape[0]

      image_batch_lr_here = x_test_lr[lo:hi]
      gen_img = generator.predict(image_batch_lr_here)
      generated_image.append(denormalize(gen_img))



    examples = x_test_lr.shape[0]
    value = randint(0, examples)
    image_batch_lr = denormalize(x_test_lr)
    
    for index in range(examples):

        value_i = index // 8
        value_j = index % 8
    
    
        plt.imshow(generated_image[index_i][index_j], interpolation='nearest')
        plt.axis('off')
        
        plt.tight_layout()
        plt.savefig(output_dir + 'high_res_result_image_%d.png' % index)
    
   

Using TensorFlow backend.


In [ ]:
# network architecture


from keras.layers import Dense
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Flatten
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import add

# Residual block
def res_block_gen(model, kernal_size, filters, strides):
    
    gen = model
    
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
        
    model = add([gen, model])
    
    return model
    
    
def up_sampling_block(model, kernal_size, filters, strides):
    
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same")(model)
    model = UpSampling2D(size = 2)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    
    return model


def discriminator_block(model, filters, kernel_size, strides):
    
    model = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model = LeakyReLU(alpha = 0.2)(model)
    
    return model

class Generator(object):

    def __init__(self, noise_shape):
        
        self.noise_shape = noise_shape

    def generator(self):
        
	    gen_input = Input(shape = self.noise_shape)
	    
	    model = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same")(gen_input)
	    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
	    
	    gen_model = model
        
      # Using 16 Residual Blocks
	    for index in range(16):
	        model = res_block_gen(model, 3, 64, 1)
	    
	    model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(model)
	    model = BatchNormalization(momentum = 0.5)(model)
	    model = add([gen_model, model])
	    
	    # Using 2 UpSampling Blocks
	    for index in range(2):
	        model = up_sampling_block(model, 3, 256, 1)
	    
	    model = Conv2D(filters = 3, kernel_size = 9, strides = 1, padding = "same")(model)
	    model = Activation('tanh')(model)
	   
	    generator_model = Model(inputs = gen_input, outputs = model)
     
        
	    return generator_model

class Discriminator(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape
    
    def discriminator(self):
        
        dis_input = Input(shape = self.image_shape)
        
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(dis_input)
        model = LeakyReLU(alpha = 0.2)(model)
        
        model = discriminator_block(model, 64, 3, 2)
        model = discriminator_block(model, 128, 3, 1)
        model = discriminator_block(model, 128, 3, 2)
        model = discriminator_block(model, 256, 3, 1)
        model = discriminator_block(model, 256, 3, 2)
        model = discriminator_block(model, 512, 3, 1)
        model = discriminator_block(model, 512, 3, 2)
        
        model = Flatten()(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha = 0.2)(model)
       
        model = Dense(1)(model)
        model = Activation('sigmoid')(model) 
        
        discriminator_model = Model(inputs = dis_input, outputs = model)
        
        return discriminator_model




In [ ]:
# optimizer, loss parameters

from keras.applications.vgg19 import VGG19
import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam

class VGG_LOSS(object):

    def __init__(self, image_shape):
        
        self.image_shape = image_shape

    # computes VGG loss or content loss
    def vgg_loss(self, y_true, y_pred):
    
        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=self.image_shape)
        vgg19.trainable = False
        # Make trainable as False
        for l in vgg19.layers:
            l.trainable = False
        model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        model.trainable = False
    
        return K.mean(K.square(model(y_true) - model(y_pred)))
    
def get_optimizer():
 
    adam = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    return adam

In [ ]:
# training data

#from tf.compat.v1.keras.backend.set_session import set_session
from keras.models import Model, load_model, save_model
from keras.layers import Input
from tqdm import tqdm
import numpy as np
import argparse

#global sess
#global graph

np.random.seed(10)

downscale_factor = 4
# Remember to change image shape if you are having different size of images
image_shape = (320,240,3)



# Combined network
def get_gan_network(discriminator, shape, generator, optimizer, vgg_loss):
    discriminator.trainable = False
    gan_input = Input(shape=shape)
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=[x,gan_output])
    gan.compile(loss=[vgg_loss, "binary_crossentropy"],
                loss_weights=[1., 1e-3],
                optimizer=optimizer)

    return gan




def train(epochs, batch_size, input_dir, output_dir, model_save_dir, number_of_images, train_test_ratio):
    
    x_train_lr, x_train_hr, x_test_lr, x_test_hr = load_training_data(input_dir, '.jpg', number_of_images, train_test_ratio) 
    loss = VGG_LOSS(image_shape)  
    
    batch_count = int(x_train_hr.shape[0] / batch_size)
    shape = (image_shape[0]//downscale_factor, image_shape[1]//downscale_factor, image_shape[2])

    #sess = tf.compat.v1.Session()
    #graph = tf.compat.v1.get_default_graph()

    
    loss = VGG_LOSS(shape)  

    epp = 2550

    #with graph.as_default():
      #tf.compat.v1.keras.backend.set_session(sess)
    generator = load_model('/content/model/gen_model%d.h5' % epp , custom_objects={'vgg_loss': loss.vgg_loss})
      #generator = Generator(shape).generator()
    
    loss = VGG_LOSS(image_shape)  

    #with graph.as_default():
      #tf.compat.v1.keras.backend.set_session(sess)
    discriminator = load_model('/content/model/dis_model%d.h5' % epp , custom_objects={'vgg_loss': loss.vgg_loss})
      #discriminator = Discriminator(image_shape).discriminator()

    optimizer = get_optimizer()
    generator.compile(loss=loss.vgg_loss, optimizer=optimizer)
    discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)
    
    gan = get_gan_network(discriminator, shape, generator, optimizer, loss.vgg_loss)
    
    loss_file = open(model_save_dir + 'losses.txt' , 'w+')
    loss_file.close()

    for e in range(epp + 1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        discriminator_loss = 0.0
        gan_loss = 0.0
        for _ in tqdm(range(batch_count)):

            #print('----****----' )

            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]
            #print(image_batch_lr.shape[0],' ',image_batch_lr.shape[1],' ',image_batch_lr.shape[2])
            #with graph.as_default():
              #tf.compat.v1.keras.backend.set_session(sess)
            generated_images_sr = generator.predict(image_batch_lr)

            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2

            fake_data_Y = np.random.random_sample(batch_size)*0.2

            #print(generated_images_sr.shape[0],' ',generated_images_sr.shape[1],' ',generated_images_sr.shape[2])
            #print(fake_data_Y.shape[0])

            
            discriminator.trainable = True
            
            #with graph.as_default():
              #tf.compat.v1.keras.backend.set_session(sess)
              #model.predict(...)
            d_loss_real = discriminator.train_on_batch(image_batch_hr, real_data_Y)
            #with graph.as_default():
              #tf.compat.v1.keras.backend.set_session(sess)
              #model.predict(...)
            d_loss_fake = discriminator.train_on_batch(generated_images_sr, fake_data_Y)
            discriminator_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
            
            rand_nums = np.random.randint(0, x_train_hr.shape[0], size=batch_size)
            image_batch_hr = x_train_hr[rand_nums]
            image_batch_lr = x_train_lr[rand_nums]

            gan_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            discriminator.trainable = False
            gan_loss = gan.train_on_batch(image_batch_lr, [image_batch_hr,gan_Y])
            
            
        print("discriminator_loss : %f" % discriminator_loss)
        print("gan_loss : ", gan_loss)
        gan_loss = str(gan_loss)
        
        loss_file = open(model_save_dir + 'losses.txt' , 'a')
        loss_file.write('epoch%d : gan_loss = %s ; discriminator_loss = %f\n' %(e, gan_loss, discriminator_loss) )
        loss_file.close()

        if e == 1 or e % 5 == 0:
            plot_generated_images(output_dir, e, generator, x_test_hr, x_test_lr)
        if e:
            #tf.compat.v1.enable_eager_execution()
            #with graph.as_default():
             # tf.compat.v1.keras.backend.set_session(sess)
            save_model(generator ,model_save_dir + 'gen_model%d.h5' % e)
            #with graph.as_default():
             # tf.compat.v1.keras.backend.set_session(sess)
            save_model(discriminator ,model_save_dir + 'dis_model%d.h5' % e)
            #tf.compat.v1.disable_eager_execution()


In [ ]:
epochs = 4000
batch_size = 8
input_dir = '/content/cocodataset/train2017'
output_dir = '/content/cocodataset/train2017_output/'
model_save_dir = '/content/model/'
number_of_images = 2000
train_test_ratio = 0.98
train(epochs, batch_size, input_dir, output_dir, model_save_dir, number_of_images, train_test_ratio)


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
















80142336/80134624 [==============================] - 6s 0us/step



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
  0%|          | 0/245 [00:00<?, ?it/s]

--------------- Epoch 2535 ---------------


100%|██████████| 245/245 [04:11<00:00,  1.03s/it]


...

100%|██████████| 245/245 [03:41<00:00,  1.10it/s]


discriminator_loss : 7.850189
gan_loss :  [0.0034679845, 0.0017323288, 1.7356557]


  0%|          | 0/245 [00:00<?, ?it/s]

--------------- Epoch 2551 ---------------


  2%|▏         | 6/245 [00:05<03:37,  1.10it/s]

In [ ]:
val_input_dir = '/content/cocodataset/val2017'
val_output_dir = '/content/cocodataset/val2017_output/'

[x_test_lr, x_test_hr] = load_test_data_for_model(val_input_dir, '.jpg', 100)

plot_test_generated_images_for_model(val_output_dir, generator, x_test_hr, x_test_lr)


In [ ]:
test_input_dir = '/content/cocodataset/test2017'
test_output_dir = '/content/cocodataset/test2017_output/'


x_test_lr = load_test_data(test_input_dir, '.jpg', number_of_images = 100)

plot_test_generated_images(test_output_dir, generator, x_test_lr)



In [ ]:
model_save_dir = '/content/model/'

image_shape = (80,60,3)


loss = VGG_LOSS(image_shape)  
model = load_model(model_save_dir + 'gen_model2000.h5' , custom_objects={'vgg_loss': loss.vgg_loss})

val_input_dir = '/content/cocodataset/val2017'
val_output_dir = '/content/val2017_output/'

[x_test_lr, x_test_hr] = load_test_data_for_model(val_input_dir, '.jpg', 100)

plot_test_generated_images_for_model(val_output_dir, model, x_test_hr, x_test_lr)


test_input_dir = '/content/cocodataset/test2017'
test_output_dir = '/content/cocodataset/test2017_output/'


x_test_lr = load_test_data(test_input_dir, '.jpg', number_of_images = 100)

plot_test_generated_images(test_output_dir, model, x_test_lr)



NameError: ignored